# Article Page Views API Example
This example illustrates how to access page view data using the [Wikimedia REST API](https://www.mediawiki.org/wiki/Wikimedia_REST_API). This example shows how to request monthly counts of page views for one specific article. The API documentation, [pageviews/per-article](https://wikimedia.org/api/rest_v1/#/Pageviews%20data), covers additional details that may be helpful when trying to use or understand this example.

## License
This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - May 5, 2022



In [15]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests
import pandas as pd

The example relies on some constants that help make the code a bit more readable.

In [23]:
#########
#
#    CONSTANTS
#

# The REST API 'pageviews' URL - this is the common URL/endpoint for all 'pageviews' API requests
API_REQUEST_PAGEVIEWS_ENDPOINT = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'

# This is a parameterized string that specifies what kind of pageviews request we are going to make
# In this case it will be a 'per-article' based request. The string is a format string so that we can
# replace each parameter with an appropriate value before making the request
API_REQUEST_PER_ARTICLE_PARAMS = 'per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}'

# The Pageviews API asks that we not exceed 100 requests per second, we add a small delay to each request
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making a request to the Wikimedia API they ask that you include a "unique ID" that will allow them to
# contact you if something happens - such as - your code exceeding request limits - or some other error happens
REQUEST_HEADERS = {
    'User-Agent': '<yangj98@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2022',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
# ARTICLE_TITLES = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

#----Create the article titles ----
df = pd.read_csv("dinosaur_genera.cleaned.SEPT.2022 - dinosaur_genera.cleaned.SEPT.2022.csv")
ARTICLE_TITLES = df['name']


# This template is used to map parameter values into the API_REQUST_PER_ARTICLE_PARAMS portion of an API request. The dictionary has a
# field/key for each of the required parameters. In the example, below, we only vary the article name, so the majority of the fields
# can stay constant for each request. Of course, these values *could* be changed if necessary.
ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "desktop",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015010100",
    "end":         "2022093000"    # this is likely the wrong end date
}


The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title.

In [29]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageviews_per_article(article_title = None, 
                                  endpoint_url = API_REQUEST_PAGEVIEWS_ENDPOINT, 
                                  endpoint_params = API_REQUEST_PER_ARTICLE_PARAMS, 
                                  request_template = ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE,
                                  headers = REQUEST_HEADERS):
    # Make sure we have an article title
    if not article_title: return None
    
    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    article_title_encoded = urllib.parse.quote(article_title.replace(' ','_'))
    request_template['article'] = article_title_encoded
    print("doing: ", request_template['article'])
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

#-----Function that creates the dataframe for the associated access and article ----#

def create_dataframe(filename, ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE):
    output_df = pd.DataFrame()
    for i in range(1, len(ARTICLE_TITLES)):
        print("Getting pageview data for: ",ARTICLE_TITLES[i])
        print("Percentage done: ", i/len(ARTICLE_TITLES))
        views = request_pageviews_per_article(ARTICLE_TITLES[i], request_template= ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE)
        views_df = pd.DataFrame(views['items'])
        output_df = pd.concat([output_df, views_df])
    #     print(views_df)

    result = output_df.to_json(orient="records")
    parsed = json.loads(result)
    print(output_df)

    f = open(filename, 'w')
    f.write(json.dumps(parsed, indent=4))
    f.close()
    
    print("Wrote to file!")
    return output_df


In [35]:
#-------------STEP 1 -----------------

#--------- GET THE DESKTOP JSON FILE ------------#

ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "desktop",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015010100",
    "end":         "2022093000"    # this is likely the wrong end date
}


df_desk = create_dataframe('dino_monthly_desktop_201501-202209.json', ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE)

Getting pageview data for:  Aachenosaurus
Percentage done:  0.0007027406886858749
doing:  Aachenosaurus
Getting pageview data for:  Aardonyx
Percentage done:  0.0014054813773717498
doing:  Aardonyx
Getting pageview data for:  Abdarainurus
Percentage done:  0.0021082220660576245
doing:  Abdarainurus
Getting pageview data for:  Abditosaurus
Percentage done:  0.0028109627547434997
doing:  Abditosaurus
Getting pageview data for:  Abelisaurus
Percentage done:  0.0035137034434293743
doing:  Abelisaurus
Getting pageview data for:  Abrictosaurus
Percentage done:  0.004216444132115249
doing:  Abrictosaurus
Getting pageview data for:  Abrosaurus
Percentage done:  0.004919184820801124
doing:  Abrosaurus
Getting pageview data for:  Abydosaurus
Percentage done:  0.005621925509486999
doing:  Abydosaurus
Getting pageview data for:  Acantholipan
Percentage done:  0.006324666198172874
doing:  Acantholipan
Getting pageview data for:  Acanthopholis
Percentage done:  0.007027406886858749
doing:  Acanthoph

Getting pageview data for:  Andesaurus
Percentage done:  0.059030217849613494
doing:  Andesaurus
Getting pageview data for:  Angolatitan
Percentage done:  0.059732958538299366
doing:  Angolatitan
Getting pageview data for:  Angulomastacator
Percentage done:  0.060435699226985246
doing:  Angulomastacator
Getting pageview data for:  Anhuilong
Percentage done:  0.06113843991567112
doing:  Anhuilong
Getting pageview data for:  Aniksosaurus
Percentage done:  0.06184118060435699
doing:  Aniksosaurus
Getting pageview data for:  Animantarx
Percentage done:  0.06254392129304287
doing:  Animantarx
Getting pageview data for:  Ankistrodon
Percentage done:  0.06324666198172874
doing:  Ankistrodon
Getting pageview data for:  Ankylosauria
Percentage done:  0.06394940267041462
doing:  Ankylosauria
Getting pageview data for:  Ankylosaurus
Percentage done:  0.0646521433591005
doing:  Ankylosaurus
Getting pageview data for:  Anodontosaurus
Percentage done:  0.06535488404778636
doing:  Anodontosaurus
Gett

Getting pageview data for:  Avisaurus
Percentage done:  0.11735769501054111
doing:  Avisaurus
Getting pageview data for:  Azendohsaurus
Percentage done:  0.11806043569922699
doing:  Azendohsaurus
Getting pageview data for:  Azhdarchoidea
Percentage done:  0.11876317638791287
doing:  Azhdarchoidea
Getting pageview data for:  Baalsaurus
Percentage done:  0.11946591707659873
doing:  Baalsaurus
Getting pageview data for:  Bactrosaurus
Percentage done:  0.12016865776528461
doing:  Bactrosaurus
Getting pageview data for:  Bagaceratops
Percentage done:  0.12087139845397049
doing:  Bagaceratops
Getting pageview data for:  Bagaraatan
Percentage done:  0.12157413914265636
doing:  Bagaraatan
Getting pageview data for:  Bagualia
Percentage done:  0.12227687983134224
doing:  Bagualia
Getting pageview data for:  Bagualosaurus
Percentage done:  0.12297962052002812
doing:  Bagualosaurus
Getting pageview data for:  Bahariasaurus
Percentage done:  0.12368236120871398
doing:  Bahariasaurus
Getting pagevi

Getting pageview data for:  Camposaurus
Percentage done:  0.1770906535488405
doing:  Camposaurus
Getting pageview data for:  Camptosaurus
Percentage done:  0.17779339423752635
doing:  Camptosaurus
Getting pageview data for:  Campylodoniscus
Percentage done:  0.17849613492621222
doing:  Campylodoniscus
Getting pageview data for:  Canardia
Percentage done:  0.1791988756148981
doing:  Canardia
Getting pageview data for:  Carcharodontosaurus
Percentage done:  0.17990161630358398
doing:  Carcharodontosaurus
Getting pageview data for:  Cardiodon
Percentage done:  0.18060435699226984
doing:  Cardiodon
Getting pageview data for:  Carnotaurus
Percentage done:  0.18130709768095574
doing:  Carnotaurus
Getting pageview data for:  Caseosaurus
Percentage done:  0.1820098383696416
doing:  Caseosaurus
Getting pageview data for:  Cathartesaura
Percentage done:  0.18271257905832747
doing:  Cathartesaura
Getting pageview data for:  Caudipteryx
Percentage done:  0.18341531974701336
doing:  Caudipteryx
Get

Getting pageview data for:  Cruxicheiros
Percentage done:  0.23471539002108222
doing:  Cruxicheiros
Getting pageview data for:  Cryolophosaurus
Percentage done:  0.2354181307097681
doing:  Cryolophosaurus
Getting pageview data for:  Cryptosaurus
Percentage done:  0.23612087139845397
doing:  Cryptosaurus
Getting pageview data for:  Cumnoria
Percentage done:  0.23682361208713984
doing:  Cumnoria
Getting pageview data for:  Daanosaurus
Percentage done:  0.23752635277582573
doing:  Daanosaurus
Getting pageview data for:  Dacentrurus
Percentage done:  0.2382290934645116
doing:  Dacentrurus
Getting pageview data for:  Daemonosaurus
Percentage done:  0.23893183415319746
doing:  Daemonosaurus
Getting pageview data for:  Dahalokely
Percentage done:  0.23963457484188336
doing:  Dahalokely
Getting pageview data for:  Dakosaurus
Percentage done:  0.24033731553056922
doing:  Dakosaurus
Getting pageview data for:  Dakotadon
Percentage done:  0.2410400562192551
doing:  Dakotadon
Getting pageview data

...

Getting pageview data for:  Sanpasaurus
Percentage done:  0.763879128601546
doing:  Sanpasaurus
Getting pageview data for:  Santanaraptor
Percentage done:  0.7645818692902319
doing:  Santanaraptor
Getting pageview data for:  Sanxiasaurus
Percentage done:  0.7652846099789178
doing:  Sanxiasaurus
Getting pageview data for:  Sarahsaurus
Percentage done:  0.7659873506676037
doing:  Sarahsaurus
Getting pageview data for:  Sarcolestes
Percentage done:  0.7666900913562895
doing:  Sarcolestes
Getting pageview data for:  Sarcosaurus
Percentage done:  0.7673928320449754
doing:  Sarcosaurus
Getting pageview data for:  Sarmientosaurus
Percentage done:  0.7680955727336612
doing:  Sarmientosaurus
Getting pageview data for:  Saurolophus
Percentage done:  0.7687983134223472
doing:  Saurolophus
Getting pageview data for:  Sauroniops
Percentage done:  0.769501054111033
doing:  Sauroniops
Getting pageview data for:  Sauropelta
Percentage done:  0.7702037947997189
doing:  Sauropelta
Getting pageview data 

Getting pageview data for:  Sphaerotholus
Percentage done:  0.8229093464511595
doing:  Sphaerotholus
Getting pageview data for:  Sphenosaurus
Percentage done:  0.8236120871398454
doing:  Sphenosaurus
Getting pageview data for:  Sphenosuchia
Percentage done:  0.8243148278285313
doing:  Sphenosuchia
Getting pageview data for:  Sphenosuchus
Percentage done:  0.8250175685172172
doing:  Sphenosuchus
Getting pageview data for:  Spiclypeus
Percentage done:  0.825720309205903
doing:  Spiclypeus
Getting pageview data for:  Spicomellus
Percentage done:  0.8264230498945889
doing:  Spicomellus
Getting pageview data for:  Spinophorosaurus
Percentage done:  0.8271257905832747
doing:  Spinophorosaurus
Getting pageview data for:  Spinops
Percentage done:  0.8278285312719607
doing:  Spinops
Getting pageview data for:  Spinosaurus
Percentage done:  0.8285312719606466
doing:  Spinosaurus
Getting pageview data for:  Spinostropheus
Percentage done:  0.8292340126493324
doing:  Spinostropheus
Getting pagevie

Getting pageview data for:  Thecospondylus
Percentage done:  0.881939564300773
doing:  Thecospondylus
Getting pageview data for:  Theiophytalia
Percentage done:  0.8826423049894588
doing:  Theiophytalia
Getting pageview data for:  Therapsid
Percentage done:  0.8833450456781448
doing:  Therapsid
Getting pageview data for:  Therizinosaurus
Percentage done:  0.8840477863668307
doing:  Therizinosaurus
Getting pageview data for:  Theropoda
Percentage done:  0.8847505270555165
doing:  Theropoda
Getting pageview data for:  Thescelosaurus
Percentage done:  0.8854532677442024
doing:  Thescelosaurus
Getting pageview data for:  Thespesius
Percentage done:  0.8861560084328882
doing:  Thespesius
Getting pageview data for:  Tianchisaurus
Percentage done:  0.8868587491215741
doing:  Tianchisaurus
Getting pageview data for:  Tianyulong
Percentage done:  0.8875614898102601
doing:  Tianyulong
Getting pageview data for:  Tianyuraptor
Percentage done:  0.8882642304989459
doing:  Tianyuraptor
Getting pagev

Getting pageview data for:  Walgettosuchus
Percentage done:  0.9409697821503865
doing:  Walgettosuchus
Getting pageview data for:  Wamweracaudia
Percentage done:  0.9416725228390724
doing:  Wamweracaudia
Getting pageview data for:  Wannanosaurus
Percentage done:  0.9423752635277582
doing:  Wannanosaurus
Getting pageview data for:  Weewarrasaurus
Percentage done:  0.9430780042164442
doing:  Weewarrasaurus
Getting pageview data for:  Wellnhoferia
Percentage done:  0.94378074490513
doing:  Wellnhoferia
Getting pageview data for:  Wendiceratops
Percentage done:  0.9444834855938159
doing:  Wendiceratops
Getting pageview data for:  Wiehenvenator
Percentage done:  0.9451862262825017
doing:  Wiehenvenator
Getting pageview data for:  Willinakaqe
Percentage done:  0.9458889669711876
doing:  Willinakaqe
Getting pageview data for:  Wintonotitan
Percentage done:  0.9465917076598735
doing:  Wintonotitan
Getting pageview data for:  Wuerhosaurus
Percentage done:  0.9472944483485594
doing:  Wuerhosauru

         project        article granularity   timestamp   access agent  views
0   en.wikipedia  Aachenosaurus     monthly  2015070100  desktop  user   2671
1   en.wikipedia  Aachenosaurus     monthly  2015080100  desktop  user  13113
2   en.wikipedia  Aachenosaurus     monthly  2015090100  desktop  user  34235
3   en.wikipedia  Aachenosaurus     monthly  2015100100  desktop  user   5361
4   en.wikipedia  Aachenosaurus     monthly  2015110100  desktop  user  13502
..           ...            ...         ...         ...      ...   ...    ...
82  en.wikipedia           Zuul     monthly  2022050100  desktop  user   1091
83  en.wikipedia           Zuul     monthly  2022060100  desktop  user    903
84  en.wikipedia           Zuul     monthly  2022070100  desktop  user   1267
85  en.wikipedia           Zuul     monthly  2022080100  desktop  user    985
86  en.wikipedia           Zuul     monthly  2022090100  desktop  user    872

[112610 rows x 7 columns]
Wrote to file!


In [30]:
#--------- GET THE MOBILE WEB JSON FILE ------------#

ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "mobile-web",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015010100",
    "end":         "2022093000"    # this is likely the wrong end date
}


df_web = create_dataframe('dino_monthly_mobileweb_201501-202209.json', ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE)






Getting pageview data for:  Aachenosaurus
Percentage done:  0.0007027406886858749
doing:  Aachenosaurus
Getting pageview data for:  Aardonyx
Percentage done:  0.0014054813773717498
doing:  Aardonyx
Getting pageview data for:  Abdarainurus
Percentage done:  0.0021082220660576245
doing:  Abdarainurus
Getting pageview data for:  Abditosaurus
Percentage done:  0.0028109627547434997
doing:  Abditosaurus
Getting pageview data for:  Abelisaurus
Percentage done:  0.0035137034434293743
doing:  Abelisaurus
Getting pageview data for:  Abrictosaurus
Percentage done:  0.004216444132115249
doing:  Abrictosaurus
Getting pageview data for:  Abrosaurus
Percentage done:  0.004919184820801124
doing:  Abrosaurus
Getting pageview data for:  Abydosaurus
Percentage done:  0.005621925509486999
doing:  Abydosaurus
Getting pageview data for:  Acantholipan
Percentage done:  0.006324666198172874
doing:  Acantholipan
Getting pageview data for:  Acanthopholis
Percentage done:  0.007027406886858749
doing:  Acanthoph

Getting pageview data for:  Anhuilong
Percentage done:  0.06113843991567112
doing:  Anhuilong
Getting pageview data for:  Aniksosaurus
Percentage done:  0.06184118060435699
doing:  Aniksosaurus
Getting pageview data for:  Animantarx
Percentage done:  0.06254392129304287
doing:  Animantarx
Getting pageview data for:  Ankistrodon
Percentage done:  0.06324666198172874
doing:  Ankistrodon
Getting pageview data for:  Ankylosauria
Percentage done:  0.06394940267041462
doing:  Ankylosauria
Getting pageview data for:  Ankylosaurus
Percentage done:  0.0646521433591005
doing:  Ankylosaurus
Getting pageview data for:  Anodontosaurus
Percentage done:  0.06535488404778636
doing:  Anodontosaurus
Getting pageview data for:  Anomalipes
Percentage done:  0.06605762473647224
doing:  Anomalipes
Getting pageview data for:  Anoplosaurus
Percentage done:  0.06676036542515812
doing:  Anoplosaurus
Getting pageview data for:  Anserimimus
Percentage done:  0.06746310611384398
doing:  Anserimimus
Getting pagevie

Getting pageview data for:  Baalsaurus
Percentage done:  0.11946591707659873
doing:  Baalsaurus
Getting pageview data for:  Bactrosaurus
Percentage done:  0.12016865776528461
doing:  Bactrosaurus
Getting pageview data for:  Bagaceratops
Percentage done:  0.12087139845397049
doing:  Bagaceratops
Getting pageview data for:  Bagaraatan
Percentage done:  0.12157413914265636
doing:  Bagaraatan
Getting pageview data for:  Bagualia
Percentage done:  0.12227687983134224
doing:  Bagualia
Getting pageview data for:  Bagualosaurus
Percentage done:  0.12297962052002812
doing:  Bagualosaurus
Getting pageview data for:  Bahariasaurus
Percentage done:  0.12368236120871398
doing:  Bahariasaurus
Getting pageview data for:  Bainoceratops
Percentage done:  0.12438510189739986
doing:  Bainoceratops
Getting pageview data for:  Bajadasaurus
Percentage done:  0.12508784258608574
doing:  Bajadasaurus
Getting pageview data for:  Balaur bondoc
Percentage done:  0.1257905832747716
doing:  Balaur_bondoc
Getting p

Getting pageview data for:  Campylodoniscus
Percentage done:  0.17849613492621222
doing:  Campylodoniscus
Getting pageview data for:  Canardia
Percentage done:  0.1791988756148981
doing:  Canardia
Getting pageview data for:  Carcharodontosaurus
Percentage done:  0.17990161630358398
doing:  Carcharodontosaurus
Getting pageview data for:  Cardiodon
Percentage done:  0.18060435699226984
doing:  Cardiodon
Getting pageview data for:  Carnotaurus
Percentage done:  0.18130709768095574
doing:  Carnotaurus
Getting pageview data for:  Caseosaurus
Percentage done:  0.1820098383696416
doing:  Caseosaurus
Getting pageview data for:  Cathartesaura
Percentage done:  0.18271257905832747
doing:  Cathartesaura
Getting pageview data for:  Caudipteryx
Percentage done:  0.18341531974701336
doing:  Caudipteryx
Getting pageview data for:  Cedarosaurus
Percentage done:  0.18411806043569923
doing:  Cedarosaurus
Getting pageview data for:  Cedarpelta
Percentage done:  0.1848208011243851
doing:  Cedarpelta
Getti

Getting pageview data for:  Cumnoria
Percentage done:  0.23682361208713984
doing:  Cumnoria
Getting pageview data for:  Daanosaurus
Percentage done:  0.23752635277582573
doing:  Daanosaurus
Getting pageview data for:  Dacentrurus
Percentage done:  0.2382290934645116
doing:  Dacentrurus
Getting pageview data for:  Daemonosaurus
Percentage done:  0.23893183415319746
doing:  Daemonosaurus
Getting pageview data for:  Dahalokely
Percentage done:  0.23963457484188336
doing:  Dahalokely
Getting pageview data for:  Dakosaurus
Percentage done:  0.24033731553056922
doing:  Dakosaurus
Getting pageview data for:  Dakotadon
Percentage done:  0.2410400562192551
doing:  Dakotadon
Getting pageview data for:  Dakotaraptor
Percentage done:  0.24174279690794098
doing:  Dakotaraptor
Getting pageview data for:  Daliansaurus
Percentage done:  0.24244553759662685
doing:  Daliansaurus
Getting pageview data for:  Dandakosaurus
Percentage done:  0.2431482782853127
doing:  Dandakosaurus
Getting pageview data for

...

Getting pageview data for:  Sanpasaurus
Percentage done:  0.763879128601546
doing:  Sanpasaurus
Getting pageview data for:  Santanaraptor
Percentage done:  0.7645818692902319
doing:  Santanaraptor
Getting pageview data for:  Sanxiasaurus
Percentage done:  0.7652846099789178
doing:  Sanxiasaurus
Getting pageview data for:  Sarahsaurus
Percentage done:  0.7659873506676037
doing:  Sarahsaurus
Getting pageview data for:  Sarcolestes
Percentage done:  0.7666900913562895
doing:  Sarcolestes
Getting pageview data for:  Sarcosaurus
Percentage done:  0.7673928320449754
doing:  Sarcosaurus
Getting pageview data for:  Sarmientosaurus
Percentage done:  0.7680955727336612
doing:  Sarmientosaurus
Getting pageview data for:  Saurolophus
Percentage done:  0.7687983134223472
doing:  Saurolophus
Getting pageview data for:  Sauroniops
Percentage done:  0.769501054111033
doing:  Sauroniops
Getting pageview data for:  Sauropelta
Percentage done:  0.7702037947997189
doing:  Sauropelta
Getting pageview data 

Getting pageview data for:  Sphaerotholus
Percentage done:  0.8229093464511595
doing:  Sphaerotholus
Getting pageview data for:  Sphenosaurus
Percentage done:  0.8236120871398454
doing:  Sphenosaurus
Getting pageview data for:  Sphenosuchia
Percentage done:  0.8243148278285313
doing:  Sphenosuchia
Getting pageview data for:  Sphenosuchus
Percentage done:  0.8250175685172172
doing:  Sphenosuchus
Getting pageview data for:  Spiclypeus
Percentage done:  0.825720309205903
doing:  Spiclypeus
Getting pageview data for:  Spicomellus
Percentage done:  0.8264230498945889
doing:  Spicomellus
Getting pageview data for:  Spinophorosaurus
Percentage done:  0.8271257905832747
doing:  Spinophorosaurus
Getting pageview data for:  Spinops
Percentage done:  0.8278285312719607
doing:  Spinops
Getting pageview data for:  Spinosaurus
Percentage done:  0.8285312719606466
doing:  Spinosaurus
Getting pageview data for:  Spinostropheus
Percentage done:  0.8292340126493324
doing:  Spinostropheus
Getting pagevie

Getting pageview data for:  Thecospondylus
Percentage done:  0.881939564300773
doing:  Thecospondylus
Getting pageview data for:  Theiophytalia
Percentage done:  0.8826423049894588
doing:  Theiophytalia
Getting pageview data for:  Therapsid
Percentage done:  0.8833450456781448
doing:  Therapsid
Getting pageview data for:  Therizinosaurus
Percentage done:  0.8840477863668307
doing:  Therizinosaurus
Getting pageview data for:  Theropoda
Percentage done:  0.8847505270555165
doing:  Theropoda
Getting pageview data for:  Thescelosaurus
Percentage done:  0.8854532677442024
doing:  Thescelosaurus
Getting pageview data for:  Thespesius
Percentage done:  0.8861560084328882
doing:  Thespesius
Getting pageview data for:  Tianchisaurus
Percentage done:  0.8868587491215741
doing:  Tianchisaurus
Getting pageview data for:  Tianyulong
Percentage done:  0.8875614898102601
doing:  Tianyulong
Getting pageview data for:  Tianyuraptor
Percentage done:  0.8882642304989459
doing:  Tianyuraptor
Getting pagev

Getting pageview data for:  Walgettosuchus
Percentage done:  0.9409697821503865
doing:  Walgettosuchus
Getting pageview data for:  Wamweracaudia
Percentage done:  0.9416725228390724
doing:  Wamweracaudia
Getting pageview data for:  Wannanosaurus
Percentage done:  0.9423752635277582
doing:  Wannanosaurus
Getting pageview data for:  Weewarrasaurus
Percentage done:  0.9430780042164442
doing:  Weewarrasaurus
Getting pageview data for:  Wellnhoferia
Percentage done:  0.94378074490513
doing:  Wellnhoferia
Getting pageview data for:  Wendiceratops
Percentage done:  0.9444834855938159
doing:  Wendiceratops
Getting pageview data for:  Wiehenvenator
Percentage done:  0.9451862262825017
doing:  Wiehenvenator
Getting pageview data for:  Willinakaqe
Percentage done:  0.9458889669711876
doing:  Willinakaqe
Getting pageview data for:  Wintonotitan
Percentage done:  0.9465917076598735
doing:  Wintonotitan
Getting pageview data for:  Wuerhosaurus
Percentage done:  0.9472944483485594
doing:  Wuerhosauru

         project        article granularity   timestamp      access agent  \
0   en.wikipedia  Aachenosaurus     monthly  2015070100  mobile-web  user   
1   en.wikipedia  Aachenosaurus     monthly  2015080100  mobile-web  user   
2   en.wikipedia  Aachenosaurus     monthly  2015090100  mobile-web  user   
3   en.wikipedia  Aachenosaurus     monthly  2015100100  mobile-web  user   
4   en.wikipedia  Aachenosaurus     monthly  2015110100  mobile-web  user   
..           ...            ...         ...         ...         ...   ...   
82  en.wikipedia           Zuul     monthly  2022050100  mobile-web  user   
83  en.wikipedia           Zuul     monthly  2022060100  mobile-web  user   
84  en.wikipedia           Zuul     monthly  2022070100  mobile-web  user   
85  en.wikipedia           Zuul     monthly  2022080100  mobile-web  user   
86  en.wikipedia           Zuul     monthly  2022090100  mobile-web  user   

    views  
0     429  
1     397  
2     275  
3     789  
4     302  
.. 

In [31]:
#--------- GET THE MOBILE APP JSON FILE ------------#

ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "mobile-app",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015010100",
    "end":         "2022093000"    # this is likely the wrong end date
}

df_app = create_dataframe('dino_monthly_mobileapp_201501-202209.json', ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE)

Getting pageview data for:  Aachenosaurus
Percentage done:  0.0007027406886858749
doing:  Aachenosaurus
Getting pageview data for:  Aardonyx
Percentage done:  0.0014054813773717498
doing:  Aardonyx
Getting pageview data for:  Abdarainurus
Percentage done:  0.0021082220660576245
doing:  Abdarainurus
Getting pageview data for:  Abditosaurus
Percentage done:  0.0028109627547434997
doing:  Abditosaurus
Getting pageview data for:  Abelisaurus
Percentage done:  0.0035137034434293743
doing:  Abelisaurus
Getting pageview data for:  Abrictosaurus
Percentage done:  0.004216444132115249
doing:  Abrictosaurus
Getting pageview data for:  Abrosaurus
Percentage done:  0.004919184820801124
doing:  Abrosaurus
Getting pageview data for:  Abydosaurus
Percentage done:  0.005621925509486999
doing:  Abydosaurus
Getting pageview data for:  Acantholipan
Percentage done:  0.006324666198172874
doing:  Acantholipan
Getting pageview data for:  Acanthopholis
Percentage done:  0.007027406886858749
doing:  Acanthoph

Getting pageview data for:  Angolatitan
Percentage done:  0.059732958538299366
doing:  Angolatitan
Getting pageview data for:  Angulomastacator
Percentage done:  0.060435699226985246
doing:  Angulomastacator
Getting pageview data for:  Anhuilong
Percentage done:  0.06113843991567112
doing:  Anhuilong
Getting pageview data for:  Aniksosaurus
Percentage done:  0.06184118060435699
doing:  Aniksosaurus
Getting pageview data for:  Animantarx
Percentage done:  0.06254392129304287
doing:  Animantarx
Getting pageview data for:  Ankistrodon
Percentage done:  0.06324666198172874
doing:  Ankistrodon
Getting pageview data for:  Ankylosauria
Percentage done:  0.06394940267041462
doing:  Ankylosauria
Getting pageview data for:  Ankylosaurus
Percentage done:  0.0646521433591005
doing:  Ankylosaurus
Getting pageview data for:  Anodontosaurus
Percentage done:  0.06535488404778636
doing:  Anodontosaurus
Getting pageview data for:  Anomalipes
Percentage done:  0.06605762473647224
doing:  Anomalipes
Getti

Getting pageview data for:  Azhdarchoidea
Percentage done:  0.11876317638791287
doing:  Azhdarchoidea
Getting pageview data for:  Baalsaurus
Percentage done:  0.11946591707659873
doing:  Baalsaurus
Getting pageview data for:  Bactrosaurus
Percentage done:  0.12016865776528461
doing:  Bactrosaurus
Getting pageview data for:  Bagaceratops
Percentage done:  0.12087139845397049
doing:  Bagaceratops
Getting pageview data for:  Bagaraatan
Percentage done:  0.12157413914265636
doing:  Bagaraatan
Getting pageview data for:  Bagualia
Percentage done:  0.12227687983134224
doing:  Bagualia
Getting pageview data for:  Bagualosaurus
Percentage done:  0.12297962052002812
doing:  Bagualosaurus
Getting pageview data for:  Bahariasaurus
Percentage done:  0.12368236120871398
doing:  Bahariasaurus
Getting pageview data for:  Bainoceratops
Percentage done:  0.12438510189739986
doing:  Bainoceratops
Getting pageview data for:  Bajadasaurus
Percentage done:  0.12508784258608574
doing:  Bajadasaurus
Getting 

Getting pageview data for:  Campylodoniscus
Percentage done:  0.17849613492621222
doing:  Campylodoniscus
Getting pageview data for:  Canardia
Percentage done:  0.1791988756148981
doing:  Canardia
Getting pageview data for:  Carcharodontosaurus
Percentage done:  0.17990161630358398
doing:  Carcharodontosaurus
Getting pageview data for:  Cardiodon
Percentage done:  0.18060435699226984
doing:  Cardiodon
Getting pageview data for:  Carnotaurus
Percentage done:  0.18130709768095574
doing:  Carnotaurus
Getting pageview data for:  Caseosaurus
Percentage done:  0.1820098383696416
doing:  Caseosaurus
Getting pageview data for:  Cathartesaura
Percentage done:  0.18271257905832747
doing:  Cathartesaura
Getting pageview data for:  Caudipteryx
Percentage done:  0.18341531974701336
doing:  Caudipteryx
Getting pageview data for:  Cedarosaurus
Percentage done:  0.18411806043569923
doing:  Cedarosaurus
Getting pageview data for:  Cedarpelta
Percentage done:  0.1848208011243851
doing:  Cedarpelta
Getti

Getting pageview data for:  Cryptosaurus
Percentage done:  0.23612087139845397
doing:  Cryptosaurus
Getting pageview data for:  Cumnoria
Percentage done:  0.23682361208713984
doing:  Cumnoria
Getting pageview data for:  Daanosaurus
Percentage done:  0.23752635277582573
doing:  Daanosaurus
Getting pageview data for:  Dacentrurus
Percentage done:  0.2382290934645116
doing:  Dacentrurus
Getting pageview data for:  Daemonosaurus
Percentage done:  0.23893183415319746
doing:  Daemonosaurus
Getting pageview data for:  Dahalokely
Percentage done:  0.23963457484188336
doing:  Dahalokely
Getting pageview data for:  Dakosaurus
Percentage done:  0.24033731553056922
doing:  Dakosaurus
Getting pageview data for:  Dakotadon
Percentage done:  0.2410400562192551
doing:  Dakotadon
Getting pageview data for:  Dakotaraptor
Percentage done:  0.24174279690794098
doing:  Dakotaraptor
Getting pageview data for:  Daliansaurus
Percentage done:  0.24244553759662685
doing:  Daliansaurus
Getting pageview data for:

...

Getting pageview data for:  Sanxiasaurus
Percentage done:  0.7652846099789178
doing:  Sanxiasaurus
Getting pageview data for:  Sarahsaurus
Percentage done:  0.7659873506676037
doing:  Sarahsaurus
Getting pageview data for:  Sarcolestes
Percentage done:  0.7666900913562895
doing:  Sarcolestes
Getting pageview data for:  Sarcosaurus
Percentage done:  0.7673928320449754
doing:  Sarcosaurus
Getting pageview data for:  Sarmientosaurus
Percentage done:  0.7680955727336612
doing:  Sarmientosaurus
Getting pageview data for:  Saurolophus
Percentage done:  0.7687983134223472
doing:  Saurolophus
Getting pageview data for:  Sauroniops
Percentage done:  0.769501054111033
doing:  Sauroniops
Getting pageview data for:  Sauropelta
Percentage done:  0.7702037947997189
doing:  Sauropelta
Getting pageview data for:  Saurophaganax
Percentage done:  0.7709065354884048
doing:  Saurophaganax
Getting pageview data for:  Sauroplites
Percentage done:  0.7716092761770906
doing:  Sauroplites
Getting pageview data

Getting pageview data for:  Sphenosaurus
Percentage done:  0.8236120871398454
doing:  Sphenosaurus
Getting pageview data for:  Sphenosuchia
Percentage done:  0.8243148278285313
doing:  Sphenosuchia
Getting pageview data for:  Sphenosuchus
Percentage done:  0.8250175685172172
doing:  Sphenosuchus
Getting pageview data for:  Spiclypeus
Percentage done:  0.825720309205903
doing:  Spiclypeus
Getting pageview data for:  Spicomellus
Percentage done:  0.8264230498945889
doing:  Spicomellus
Getting pageview data for:  Spinophorosaurus
Percentage done:  0.8271257905832747
doing:  Spinophorosaurus
Getting pageview data for:  Spinops
Percentage done:  0.8278285312719607
doing:  Spinops
Getting pageview data for:  Spinosaurus
Percentage done:  0.8285312719606466
doing:  Spinosaurus
Getting pageview data for:  Spinostropheus
Percentage done:  0.8292340126493324
doing:  Spinostropheus
Getting pageview data for:  Spinosuchus
Percentage done:  0.8299367533380183
doing:  Spinosuchus
Getting pageview da

Getting pageview data for:  Thecospondylus
Percentage done:  0.881939564300773
doing:  Thecospondylus
Getting pageview data for:  Theiophytalia
Percentage done:  0.8826423049894588
doing:  Theiophytalia
Getting pageview data for:  Therapsid
Percentage done:  0.8833450456781448
doing:  Therapsid
Getting pageview data for:  Therizinosaurus
Percentage done:  0.8840477863668307
doing:  Therizinosaurus
Getting pageview data for:  Theropoda
Percentage done:  0.8847505270555165
doing:  Theropoda
Getting pageview data for:  Thescelosaurus
Percentage done:  0.8854532677442024
doing:  Thescelosaurus
Getting pageview data for:  Thespesius
Percentage done:  0.8861560084328882
doing:  Thespesius
Getting pageview data for:  Tianchisaurus
Percentage done:  0.8868587491215741
doing:  Tianchisaurus
Getting pageview data for:  Tianyulong
Percentage done:  0.8875614898102601
doing:  Tianyulong
Getting pageview data for:  Tianyuraptor
Percentage done:  0.8882642304989459
doing:  Tianyuraptor
Getting pagev

Getting pageview data for:  Walgettosuchus
Percentage done:  0.9409697821503865
doing:  Walgettosuchus
Getting pageview data for:  Wamweracaudia
Percentage done:  0.9416725228390724
doing:  Wamweracaudia
Getting pageview data for:  Wannanosaurus
Percentage done:  0.9423752635277582
doing:  Wannanosaurus
Getting pageview data for:  Weewarrasaurus
Percentage done:  0.9430780042164442
doing:  Weewarrasaurus
Getting pageview data for:  Wellnhoferia
Percentage done:  0.94378074490513
doing:  Wellnhoferia
Getting pageview data for:  Wendiceratops
Percentage done:  0.9444834855938159
doing:  Wendiceratops
Getting pageview data for:  Wiehenvenator
Percentage done:  0.9451862262825017
doing:  Wiehenvenator
Getting pageview data for:  Willinakaqe
Percentage done:  0.9458889669711876
doing:  Willinakaqe
Getting pageview data for:  Wintonotitan
Percentage done:  0.9465917076598735
doing:  Wintonotitan
Getting pageview data for:  Wuerhosaurus
Percentage done:  0.9472944483485594
doing:  Wuerhosauru

         project        article granularity   timestamp      access agent  \
0   en.wikipedia  Aachenosaurus     monthly  2015070100  mobile-app  user   
1   en.wikipedia  Aachenosaurus     monthly  2015080100  mobile-app  user   
2   en.wikipedia  Aachenosaurus     monthly  2015090100  mobile-app  user   
3   en.wikipedia  Aachenosaurus     monthly  2015100100  mobile-app  user   
4   en.wikipedia  Aachenosaurus     monthly  2015110100  mobile-app  user   
..           ...            ...         ...         ...         ...   ...   
82  en.wikipedia           Zuul     monthly  2022050100  mobile-app  user   
83  en.wikipedia           Zuul     monthly  2022060100  mobile-app  user   
84  en.wikipedia           Zuul     monthly  2022070100  mobile-app  user   
85  en.wikipedia           Zuul     monthly  2022080100  mobile-app  user   
86  en.wikipedia           Zuul     monthly  2022090100  mobile-app  user   

    views  
0      14  
1      15  
2       8  
3       5  
4       9  
.. 

In [36]:
#--------mobile json ------ (need to combine two dataframes)
output_df = pd.concat([df_app, df_web])
result = output_df.to_json(orient="records")
parsed = json.loads(result)
print(output_df)

f = open('dino_monthly_mobile__201501-202209', 'w')
f.write(json.dumps(parsed, indent=4))
f.close()


#---------cumulative json------
output_df = pd.concat([df_app, df_web, df_desk])
result = output_df.to_json(orient="records")
parsed = json.loads(result)
print(output_df)

f = open('dino_monthly_cumulative__201501-202209', 'w')
f.write(json.dumps(parsed, indent=4))
f.close()

         project        article granularity   timestamp      access agent  \
0   en.wikipedia  Aachenosaurus     monthly  2015070100  mobile-app  user   
1   en.wikipedia  Aachenosaurus     monthly  2015080100  mobile-app  user   
2   en.wikipedia  Aachenosaurus     monthly  2015090100  mobile-app  user   
3   en.wikipedia  Aachenosaurus     monthly  2015100100  mobile-app  user   
4   en.wikipedia  Aachenosaurus     monthly  2015110100  mobile-app  user   
..           ...            ...         ...         ...         ...   ...   
82  en.wikipedia           Zuul     monthly  2022050100     desktop  user   
83  en.wikipedia           Zuul     monthly  2022060100     desktop  user   
84  en.wikipedia           Zuul     monthly  2022070100     desktop  user   
85  en.wikipedia           Zuul     monthly  2022080100     desktop  user   
86  en.wikipedia           Zuul     monthly  2022090100     desktop  user   

    views  
0      14  
1      15  
2       8  
3       5  
4       9  
.. 

In [72]:
df_cumulative = pd.concat([df_app, df_web, df_desk])
df_cumulative['timestamp'].unique()

df_vals = pd.DataFrame()
avg_views = []
# times = df_cumulative['timestamp'].unique()
# print(max(df_cumulative[df_cumulative['timestamp'] == '2015070100']))
for article in df_cumulative['article'].unique():
    avg_views.append(df_cumulative['views'][df_cumulative['article'] == article].mean())
    print(article)
    


Aachenosaurus
Aardonyx
Abdarainurus
Abditosaurus
Abelisaurus
Abrictosaurus
Abrosaurus
Abydosaurus
Acantholipan
Acanthopholis
Achelousaurus
Acheroraptor
Achillesaurus
Achillobator
Acristavus
Acrocanthosaurus
Acrotholus
Actiosaurus
Adamantisaurus
Adasaurus
Adelolophus
Adeopapposaurus
Adratiklit
Adynomosaurus
Aegyptosaurus
Aeolosaurus
Aepisaurus
Aepyornithomimus
Aerosteon
Afromimus
Afrovenator
Agathaumas
Aggiosaurus
Agilisaurus
Agnosphitys
Agrosaurus
Agujaceratops
Agustinia
Ahshislepelta
Ajkaceratops
Ajnabia
Akainacephalus
Alamosaurus
Alaskacephale
Albalophosaurus
Albertaceratops
Albertadromeus
Albertavenator
Albertonykus
Albertosaurus
Albinykus
Albisaurus
Alcovasaurus
Alectrosaurus
Aletopelta
Algoasaurus
Alioramus
Allosaurus
Almas_ukhaa
Alnashetri
Alocodon
Altirhinus
Altispinax
Alvarezsaurus
Alwalkeria
Alxasaurus
Amanzia
Amargasaurus
Amargatitanis
Amazonsaurus
Ambopteryx
Ampelosaurus
Amphicoelias
Amtocephale
Amtosaurus
Amurosaurus
Amygdalodon
Anabisetia
Analong
Anasazisaurus
Anchiceratop

Kakuru
Kamuysaurus
Kangnasaurus
Kansaignathus
Karongasaurus
Katepensaurus
Kayentavenator
Kazaklambia
Kelmayisaurus
Kelumapusaura
Kemkemia
Kentrosaurus
Kerberosaurus
Khaan
Kholumolumo
Khulsanurus
Kileskus
Kinnareemimus
Klamelisaurus
Kol_ghuva
Koparion
Koreaceratops
Koreanosaurus
Koshisaurus
Kosmoceratops
Kotasaurus
Koutalisaurus
Kritosaurus
Kryptops
Krzyzanowskisaurus
Kulceratops
Kulindadromeus
Kunbarrasaurus
Kundurosaurus
Kuru_kulla
Kurupi_itaata
Kuszholia
Kwanasaurus
Labocania
Laevisuchus
Lagerpeton
Lagosuchus
Laiyangosaurus
Lajasvenator
Lambeosaurus
Lametasaurus
Lamplughsaura
Lanzhousaurus
Laosaurus
Lapampasaurus
Laplatasaurus
Lapparentosaurus
Laquintasaura
Lariosaurus
Latenivenatrix
Latirhinus
Lavocatisaurus
Leaellynasaura
Ledumahadi
Leinkupal
Leonerasaurus
Lepidus_praecisio
Leptoceratops
Leshansaurus
Lesothosaurus
Lessemsaurus
Levnesovia
Lewisuchus
Lexovisaurus
Leyesaurus
Liaoceratops
Liaoningosaurus
Liaoningotitan
Liaoningvenator
Libycosaurus
Ligabueino
Ligabuesaurus
Liliensternus

Timimus
Timurlengia
Titanoceratops
Titanosaurus
Tlatolophus
Tochisaurus
Tonganosaurus
Tongtianlong
Tornieria
Torosaurus
Torvosaurus
Tototlmimus
Trachodon
Tralkasaurus
Tratayenia
Traukutitan
Trialestes
Triceratops
Trierarchuncus
Trigonosaurus
Trimucrodon
Trinisaura
Triunfosaurus
Troodon
Tsaagan
Tsagantegia
Tsintaosaurus
Tuebingosaurus
Tugulusaurus
Tuojiangosaurus
Turanoceratops
Turiasaurus
Tylocephale
Tyrannosaurus
Tyrannotitan
Uberabatitan
Ubirajara_jubatus
Udanoceratops
Ultrasaurus
Ulughbegsaurus
Unaysaurus
Unenlagia
Unescoceratops
Unquillosaurus
Urbacodon
Utahceratops
Utahraptor
Uteodon
Vagaceratops
Vahiny
Valdoraptor
Valdosaurus
Vallibonavenatrix
Variraptor
Vayuraptor
Vectaerovenator
Vectiraptor
Velafrons
Velocipes
Velociraptor
Velocisaurus
Venaticosuchus
Venenosaurus
Vespersaurus
Veterupristisaurus
Viavenator
Volgatitan
Volkheimeria
Vouivria
Vulcanodon
Wakinosaurus
Walgettosuchus
Wamweracaudia
Wannanosaurus
Weewarrasaurus
Wellnhoferia
Wendiceratops
Wiehenvenator
Willinakaqe
Wintono

In [83]:
df_vals = pd.DataFrame()
df_vals['article'] = df_cumulative['article'].unique()
df_vals['avg_views'] = avg_views

df_vals.sort_values(by=['avg_views'])

,article,avg_views
560,Honghesaurus,7.616858
898,Ornithomerus,10.191571
1200,Suchoprion,10.229885
1042,Rachitrema,10.812261
922,Palaeoctonus,11.498084
...,...,...
1187,Stegosaurus,12000.781609
1284,Triceratops,12393.352490
1178,Spinosaurus,16499.766284
1326,Velociraptor,23157.567050


In [70]:
df_cumulative['views'][df_cumulative['article'] == "Aachenosaurus"].mean()

660.448275862069

In [150]:
#-------------STEP 2 -----------------
import matplotlib.pyplot as plt

df_mobile = pd.concat([df_web, df_app])
df_cumulative = pd.concat([df_web, df_app, df_desk])


def get_min_max_df(df):
    df = df.groupby('article')
    df.first()
#     df_vals = pd.DataFrame()
#     avg_views = []
# #     article_name = []
#     # times = df_cumulative['timestamp'].unique()
#     # print(max(df_cumulative[df_cumulative['timestamp'] == '2015070100']))
#     for time in df['timestamp'].unique():
# #         article_name.append(df['article'][df['timestamp'] == time])
#         avg_views.append(df['views'][df['timestamp'] == time].mean())
        
# #         print("time done:", time)
        
#     df_vals['timestamp'] = df['timestamp'].unique()
#     df_vals['avg_views'] = avg_views

    
    return df_vals

def max_min_average(df_mobile, df_desk):
    
    df_mobile_vals = get_min_max_df(df_mobile)
    df_desktop_vals = get_min_max_df(df_desk)
#     df_mobile_vals = df_mobile_vals.sort_values(by=['avg_views'])
#     df_desktop_vals = df_desktop_vals.sort_values(by=['avg_views'])
#     plt.plot(df_mobile_vals['timestamp'], df_mobile_vals['avg_views'], label = 'mobile')
#     plt.plot(df_desktop_vals['timestamp'], df_desktop_vals['avg_views'], label = 'desktop')
#     plt.legend()
#     plt.show()
#     plt.plot(df_mobile['timestamp'], df_mobile['views'])
    
max_min_average(df_mobile, df_desk)


In [192]:
df = df_mobile
df = df.groupby(['article'])['views'].mean()
df = df.sort_values()
temp_df = df_mobile.groupby('article')
df

article
Honghesaurus           2.942529
Suchoprion             3.178161
Protecovasaurus        3.488506
Palaeoctonus           4.298851
Gigantoscelus          4.643678
                       ...     
Stegosaurus        10565.735632
Triceratops        10615.350575
Spinosaurus        14479.206897
Velociraptor       21047.045977
Tyrannosaurus      29768.850575
Name: views, Length: 1422, dtype: float64

In [117]:
df_mobile_vals

,timestamp,avg_views
0,2015070100,519.889744
1,2015080100,310.217150
2,2015090100,237.387894
3,2015100100,300.134354
4,2015110100,281.134975
...,...,...
82,2022050100,436.295053
83,2022060100,695.241696
84,2022070100,544.240113
85,2022080100,469.782394


In [257]:
df = df_mobile
def top_10_peak_page(df):
    #find the month for each article that contains peak views
    temp_df = df.sort_values('views', ascending=False).head(36)

    dinosaurs = list(temp_df['article'].unique())
    # dinosaurs
    temp_df = temp_df[temp_df['article'].isin(dinosaurs)]
    return temp_df.groupby('article').first()

In [199]:
df_mobile[df_mobile['article'] == "Aachenosaurus"].sort_values('views')

,project,article,granularity,timestamp,access,agent,views
59,en.wikipedia,Aachenosaurus,monthly,2020060100,mobile-app,user,3
3,en.wikipedia,Aachenosaurus,monthly,2015100100,mobile-app,user,5
6,en.wikipedia,Aachenosaurus,monthly,2016010100,mobile-app,user,5
16,en.wikipedia,Aachenosaurus,monthly,2016110100,mobile-app,user,6
51,en.wikipedia,Aachenosaurus,monthly,2019100100,mobile-app,user,7
...,...,...,...,...,...,...,...
38,en.wikipedia,Aachenosaurus,monthly,2018090100,mobile-web,user,513
35,en.wikipedia,Aachenosaurus,monthly,2018060100,mobile-web,user,521
36,en.wikipedia,Aachenosaurus,monthly,2018070100,mobile-web,user,687
3,en.wikipedia,Aachenosaurus,monthly,2015100100,mobile-web,user,789


In [211]:
for time in df_mobile['timestamp'].unique():
    each_df = df_mobile[df_mobile['timestamp'] == time].sort_values('views', ascending=False)
    print(each_df.head(10))

        project         article granularity   timestamp      access agent  \
0  en.wikipedia    Velociraptor     monthly  2015070100  mobile-web  user   
0  en.wikipedia   Tyrannosaurus     monthly  2015070100  mobile-web  user   
0  en.wikipedia     Spinosaurus     monthly  2015070100  mobile-web  user   
0  en.wikipedia   Dilophosaurus     monthly  2015070100  mobile-web  user   
0  en.wikipedia  Giganotosaurus     monthly  2015070100  mobile-web  user   
0  en.wikipedia     Deinonychus     monthly  2015070100  mobile-web  user   
0  en.wikipedia     Triceratops     monthly  2015070100  mobile-web  user   
0  en.wikipedia     Apatosaurus     monthly  2015070100  mobile-web  user   
0  en.wikipedia       Pterosaur     monthly  2015070100  mobile-web  user   
0  en.wikipedia     Stegosaurus     monthly  2015070100  mobile-web  user   

    views  
0  137803  
0  109161  
0   68535  
0   28783  
0   27713  
0   25113  
0   24675  
0   24145  
0   23451  
0   21683  
        project     

         project        article granularity   timestamp      access agent  \
10  en.wikipedia  Tyrannosaurus     monthly  2016050100  mobile-web  user   
10  en.wikipedia   Velociraptor     monthly  2016050100  mobile-web  user   
10  en.wikipedia       Mosasaur     monthly  2016050100  mobile-web  user   
10  en.wikipedia    Spinosaurus     monthly  2016050100  mobile-web  user   
10  en.wikipedia    Stegosaurus     monthly  2016050100  mobile-web  user   
10  en.wikipedia    Triceratops     monthly  2016050100  mobile-web  user   
10  en.wikipedia      Pterosaur     monthly  2016050100  mobile-web  user   
10  en.wikipedia   Brontosaurus     monthly  2016050100  mobile-web  user   
10  en.wikipedia     Crocodilia     monthly  2016050100  mobile-web  user   
10  en.wikipedia    Apatosaurus     monthly  2016050100  mobile-web  user   

    views  
10  75668  
10  50955  
10  29890  
10  25986  
10  20916  
10  17329  
10  16590  
10  15310  
10  14375  
10  13981  
         project    

         project         article granularity   timestamp      access agent  \
20  en.wikipedia   Tyrannosaurus     monthly  2017030100  mobile-web  user   
20  en.wikipedia    Velociraptor     monthly  2017030100  mobile-web  user   
20  en.wikipedia     Spinosaurus     monthly  2017030100  mobile-web  user   
20  en.wikipedia       Pterosaur     monthly  2017030100  mobile-web  user   
20  en.wikipedia     Stegosaurus     monthly  2017030100  mobile-web  user   
20  en.wikipedia     Triceratops     monthly  2017030100  mobile-web  user   
20  en.wikipedia    Brontosaurus     monthly  2017030100  mobile-web  user   
20  en.wikipedia  Giganotosaurus     monthly  2017030100  mobile-web  user   
20  en.wikipedia   Archaeopteryx     monthly  2017030100  mobile-web  user   
20  en.wikipedia   Brachiosaurus     monthly  2017030100  mobile-web  user   

    views  
20  56570  
20  39291  
20  28742  
20  24691  
20  20349  
20  19026  
20  18220  
20  17123  
20  16285  
20  15846  
         

         project        article granularity   timestamp      access agent  \
31  en.wikipedia  Tyrannosaurus     monthly  2018020100  mobile-web  user   
31  en.wikipedia   Velociraptor     monthly  2018020100  mobile-web  user   
31  en.wikipedia    Stegosaurus     monthly  2018020100  mobile-web  user   
31  en.wikipedia    Spinosaurus     monthly  2018020100  mobile-web  user   
31  en.wikipedia    Triceratops     monthly  2018020100  mobile-web  user   
31  en.wikipedia   Brontosaurus     monthly  2018020100  mobile-web  user   
31  en.wikipedia      Pterosaur     monthly  2018020100  mobile-web  user   
31  en.wikipedia      Theropoda     monthly  2018020100  mobile-web  user   
31  en.wikipedia     Stegoceras     monthly  2018020100  mobile-web  user   
31  en.wikipedia  Brachiosaurus     monthly  2018020100  mobile-web  user   

    views  
31  44402  
31  30510  
31  21966  
31  19379  
31  17112  
31  14536  
31  13415  
31  12617  
31  12587  
31  12168  
         project    

         project        article granularity   timestamp      access agent  \
41  en.wikipedia  Tyrannosaurus     monthly  2018120100  mobile-web  user   
41  en.wikipedia  Brachiosaurus     monthly  2018120100  mobile-web  user   
41  en.wikipedia   Velociraptor     monthly  2018120100  mobile-web  user   
41  en.wikipedia   Brontosaurus     monthly  2018120100  mobile-web  user   
41  en.wikipedia    Spinosaurus     monthly  2018120100  mobile-web  user   
41  en.wikipedia    Stegosaurus     monthly  2018120100  mobile-web  user   
41  en.wikipedia    Triceratops     monthly  2018120100  mobile-web  user   
41  en.wikipedia     Allosaurus     monthly  2018120100  mobile-web  user   
41  en.wikipedia    Deinonychus     monthly  2018120100  mobile-web  user   
41  en.wikipedia      Pterosaur     monthly  2018120100  mobile-web  user   

    views  
41  50337  
41  39321  
41  34139  
41  20924  
41  20753  
41  20666  
41  18722  
41  14165  
41  14024  
41  13993  
         project    

         project        article granularity   timestamp      access agent  \
51  en.wikipedia  Tyrannosaurus     monthly  2019100100  mobile-web  user   
51  en.wikipedia   Velociraptor     monthly  2019100100  mobile-web  user   
51  en.wikipedia    Spinosaurus     monthly  2019100100  mobile-web  user   
51  en.wikipedia    Triceratops     monthly  2019100100  mobile-web  user   
51  en.wikipedia    Stegosaurus     monthly  2019100100  mobile-web  user   
51  en.wikipedia     Crocodilia     monthly  2019100100  mobile-web  user   
51  en.wikipedia   Brontosaurus     monthly  2019100100  mobile-web  user   
51  en.wikipedia      Pterosaur     monthly  2019100100  mobile-web  user   
51  en.wikipedia  Brachiosaurus     monthly  2019100100  mobile-web  user   
51  en.wikipedia  Archaeopteryx     monthly  2019100100  mobile-web  user   

    views  
51  46279  
51  40154  
51  19199  
51  18914  
51  17859  
51  14591  
51  14473  
51  13456  
51  12520  
51  11643  
         project    

         project        article granularity   timestamp      access agent  \
61  en.wikipedia  Tyrannosaurus     monthly  2020080100  mobile-web  user   
61  en.wikipedia   Velociraptor     monthly  2020080100  mobile-web  user   
61  en.wikipedia    Spinosaurus     monthly  2020080100  mobile-web  user   
61  en.wikipedia    Triceratops     monthly  2020080100  mobile-web  user   
61  en.wikipedia  Dilophosaurus     monthly  2020080100  mobile-web  user   
61  en.wikipedia   Brontosaurus     monthly  2020080100  mobile-web  user   
61  en.wikipedia    Stegosaurus     monthly  2020080100  mobile-web  user   
61  en.wikipedia  Brachiosaurus     monthly  2020080100  mobile-web  user   
61  en.wikipedia    Deinonychus     monthly  2020080100  mobile-web  user   
61  en.wikipedia     Crocodilia     monthly  2020080100  mobile-web  user   

    views  
61  70896  
61  63105  
61  47844  
61  30472  
61  27917  
61  26273  
61  26020  
61  23645  
61  22965  
61  20325  
         project    

         project        article granularity   timestamp      access agent  \
70  en.wikipedia   Deinocheirus     monthly  2021050100  mobile-web  user   
70  en.wikipedia  Achelousaurus     monthly  2021050100  mobile-web  user   
70  en.wikipedia  Tyrannosaurus     monthly  2021050100  mobile-web  user   
70  en.wikipedia   Velociraptor     monthly  2021050100  mobile-web  user   
70  en.wikipedia    Spinosaurus     monthly  2021050100  mobile-web  user   
70  en.wikipedia    Stegosaurus     monthly  2021050100  mobile-web  user   
70  en.wikipedia   Brontosaurus     monthly  2021050100  mobile-web  user   
70  en.wikipedia    Triceratops     monthly  2021050100  mobile-web  user   
70  en.wikipedia     Crocodilia     monthly  2021050100  mobile-web  user   
70  en.wikipedia  Brachiosaurus     monthly  2021050100  mobile-web  user   

     views  
70  228922  
70  111657  
70   59167  
70   38081  
70   29411  
70   25210  
70   24787  
70   24111  
70   21492  
70   20728  
         

         project        article granularity   timestamp      access agent  \
80  en.wikipedia  Tyrannosaurus     monthly  2022030100  mobile-web  user   
80  en.wikipedia   Velociraptor     monthly  2022030100  mobile-web  user   
80  en.wikipedia  Archaeopteryx     monthly  2022030100  mobile-web  user   
80  en.wikipedia    Spinosaurus     monthly  2022030100  mobile-web  user   
80  en.wikipedia     Allosaurus     monthly  2022030100  mobile-web  user   
80  en.wikipedia  Achelousaurus     monthly  2022030100  mobile-web  user   
80  en.wikipedia  Dilophosaurus     monthly  2022030100  mobile-web  user   
80  en.wikipedia    Triceratops     monthly  2022030100  mobile-web  user   
80  en.wikipedia   Brontosaurus     monthly  2022030100  mobile-web  user   
80  en.wikipedia    Stegosaurus     monthly  2022030100  mobile-web  user   

    views  
80  58483  
80  27183  
80  26972  
80  26704  
80  25661  
80  22633  
80  22438  
80  20523  
80  19921  
80  19676  
         project    

In [255]:

#found that the top 10 articles are these by peak views:

,project,granularity,timestamp,access,agent,views
article,,,,,,
Achelousaurus,en.wikipedia,monthly,2021020100,desktop,user,205150
Argentinosaurus,en.wikipedia,monthly,2016120100,mobile-web,user,78020
Borealopelta,en.wikipedia,monthly,2019090100,mobile-web,user,115782
Deinocheirus,en.wikipedia,monthly,2021050100,mobile-web,user,228922
Giganotosaurus,en.wikipedia,monthly,2022060100,mobile-web,user,154393
Halszkaraptor,en.wikipedia,monthly,2017120100,desktop,user,103251
Nigersaurus,en.wikipedia,monthly,2020020100,mobile-web,user,233924
Therizinosaurus,en.wikipedia,monthly,2022060100,mobile-web,user,146164
Tyrannosaurus,en.wikipedia,monthly,2018060100,mobile-web,user,132621


In [266]:
temp_df = df_cumulative.sort_values('views', ascending=False).head(36)

dinosaurs = list(temp_df['article'].unique())
# dinosaurs
temp_df = temp_df[temp_df['article'].isin(dinosaurs)]
df = temp_df.groupby('article').first()
df.sort_values('views', ascending=False)

,project,granularity,timestamp,access,agent,views
article,,,,,,
Nigersaurus,en.wikipedia,monthly,2020020100,mobile-web,user,233924
Deinocheirus,en.wikipedia,monthly,2021050100,mobile-web,user,228922
Achelousaurus,en.wikipedia,monthly,2021020100,desktop,user,205150
Giganotosaurus,en.wikipedia,monthly,2022060100,mobile-web,user,154393
Therizinosaurus,en.wikipedia,monthly,2022060100,mobile-web,user,146164
Velociraptor,en.wikipedia,monthly,2015070100,mobile-web,user,137803
Tyrannosaurus,en.wikipedia,monthly,2018060100,mobile-web,user,132621
Borealopelta,en.wikipedia,monthly,2019090100,mobile-web,user,115782
Halszkaraptor,en.wikipedia,monthly,2017120100,desktop,user,103251
